In [85]:
from google.colab import drive
import pandas as pd
import numpy as np

In [86]:
drive.mount('/content/drive')
dataset = pd.read_excel('/content/drive/My Drive/FGA/Job_Description_and_Salary_in_Indonesia.xlsx')
dataset.head(5)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,id,job_title,location,salary_currency,career_level,experience_level,education_level,employment_type,job_function,job_benefits,company_process_time,company_size,company_industry,job_description,salary
0,1,Facility Maintenance & Smart Warehouse Manager,Bandung,IDR,Manajer/Asisten Manajer,5 tahun,"Sertifikat Professional, D3 (Diploma), D4 (Dip...",Penuh Waktu,"Manufaktur,Pemeliharaan",NaN,NaN,NaN,NaN,Deskripsi PekerjaanRequirements :D3/SI from re...,NaN
1,2,Procurement Department Head,Jakarta Raya,IDR,Manajer/Asisten Manajer,5 tahun,"Sarjana (S1), Diploma Pascasarjana, Gelar Prof...",Penuh Waktu,"Manufaktur,Pembelian/Manajemen Material",NaN,25 days,51 - 200 pekerja,Manajemen/Konsulting HR,Job Role: 1. Responsible for material availabi...,NaN
2,3,SALES ADMIN,Jakarta Barat,IDR,Supervisor/Koordinator,4 tahun,Sarjana (S1),Penuh Waktu,"Penjualan / Pemasaran,Penjualan Ritel","Waktu regular, Senin - Jumat;Bisnis (contoh: K...",30 days,51 - 200 pekerja,Umum & Grosir,Internal Sales & AdminJob Description :We are ...,NaN
3,4,City Operation Lead Shopee Express (Cirebon),Cirebon,IDR,Supervisor/Koordinator,5 tahun,"Sarjana (S1), Diploma Pascasarjana, Gelar Prof...",Penuh Waktu,"Pelayanan,Logistik/Rantai Pasokan","Tip;Waktu regular, Senin - Jumat;Kasual (conto...",21 days,2001 - 5000 pekerja,Retail/Merchandise,Job Description:Responsible for HSE implementa...,NaN
4,5,Japanese Interpreter,Bekasi,IDR,Pegawai (non-manajemen & non-supervisor),2 tahun,"Sertifikat Professional, D3 (Diploma), D4 (Dip...",Penuh Waktu,"Lainnya,Jurnalis/Editor",NaN,23 days,201 - 500 pekerja,Manajemen/Konsulting HR,Overview: Our clients is manufacture for autom...,NaN


In [87]:
chosen_columns = ['id','education_level','experience_level','location','employment_type']
df = dataset[chosen_columns]
df = df.applymap(lambda x: x.lower() if isinstance(x, str) else x)

<ipython-input-87-1f0233699da4>:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.lower() if isinstance(x, str) else x)


In [88]:
df.head(5)

,id,education_level,experience_level,location,employment_type
0,1,"sertifikat professional, d3 (diploma), d4 (dip...",5 tahun,bandung,penuh waktu
1,2,"sarjana (s1), diploma pascasarjana, gelar prof...",5 tahun,jakarta raya,penuh waktu
2,3,sarjana (s1),4 tahun,jakarta barat,penuh waktu
3,4,"sarjana (s1), diploma pascasarjana, gelar prof...",5 tahun,cirebon,penuh waktu
4,5,"sertifikat professional, d3 (diploma), d4 (dip...",2 tahun,bekasi,penuh waktu


In [89]:
df['experience_level'] = df['experience_level'].str.replace('lebih dari ', '')
df['experience_level'] = df['experience_level'].str.replace(' tahun', '')
df['experience_level'] = df['experience_level'].replace(np.nan, 0)
df['experience_level'] = df['experience_level'].astype(int)

In [90]:
df['education_level'] = df['education_level'].astype('category')
df['location'] = df['location'].astype('category')
df['employment_type'] = df['employment_type'].astype('category')

print(df.dtypes)

id                     int64
education_level     category
experience_level       int64
location            category
employment_type     category
dtype: object


In [91]:
df.nunique()

,0
id,34746
education_level,21
experience_level,20
location,200
employment_type,10


In [92]:
df.isnull().sum()

,0
id,0
education_level,0
experience_level,0
location,0
employment_type,1344


In [93]:
df.head(10)

,id,education_level,experience_level,location,employment_type
0,1,"sertifikat professional, d3 (diploma), d4 (dip...",5,bandung,penuh waktu
1,2,"sarjana (s1), diploma pascasarjana, gelar prof...",5,jakarta raya,penuh waktu
2,3,sarjana (s1),4,jakarta barat,penuh waktu
3,4,"sarjana (s1), diploma pascasarjana, gelar prof...",5,cirebon,penuh waktu
4,5,"sertifikat professional, d3 (diploma), d4 (dip...",2,bekasi,penuh waktu
5,6,"sertifikat professional, d3 (diploma), d4 (dip...",0,balikpapan,penuh waktu
6,7,"sma, smu/smk/stm",1,tangerang,penuh waktu
7,8,sarjana (s1),2,tangerang,penuh waktu
8,9,sarjana (s1),4,bekasi,penuh waktu
9,10,"sertifikat professional, d3 (diploma), d4 (dip...",3,banten,penuh waktu


In [95]:
def recommend_jobs(experience, education, location=None, employment_type=None):
    filtered_df = df[
        (df['experience_level'] <= experience) &
        (df['education_level'].str.contains(education, case=False))
    ]

    if location:
        filtered_df = filtered_df[filtered_df['location'].str.contains(location, case=False)]

    if employment_type:
        filtered_df = filtered_df[filtered_df['employment_type'].str.contains(employment_type, case=False)]

    recommended_jobs = filtered_df.sort_values(by='experience_level', ascending=True)

    return recommended_jobs[['id', 'location', 'experience_level', 'education_level', 'employment_type']]

recommended_jobs = recommend_jobs(3, 'sarjana', location='jakarta', employment_type='penuh waktu')
print(recommended_jobs)

          id         location  experience_level  \
8577    8578    jakarta pusat                 0   
10326  10327     jakarta raya                 0   
31426  31427    jakarta pusat                 0   
3620    3621  jakarta selatan                 0   
29789  29790    jakarta utara                 0   
...      ...              ...               ...   
10985  10986     jakarta raya                 3   
25520  25521     jakarta raya                 3   
10977  10978     jakarta raya                 3   
11042  11043     jakarta raya                 3   
9478    9479     jakarta raya                 3   

                                         education_level employment_type  
8577   sertifikat professional, d3 (diploma), d4 (dip...     penuh waktu  
10326  sertifikat professional, d3 (diploma), d4 (dip...     penuh waktu  
31426                                       sarjana (s1)     penuh waktu  
3620   sarjana (s1), diploma pascasarjana, gelar prof...     penuh waktu  
29789       